In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp
import distributed_gp as dgp

In [ ]:
domain_bounds = [('x1', 0, 6), ('x2', 0, 6)]
range_bounds = (-1, 2.5)

In [ ]:
noisy = False
def to_fit(X):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    x, y = X[:,0], X[:,1]
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    val = 2*a + b - c + 0.17 * d + 2*e
    if noisy:
        val += np.random.normal(0, 0.2, size=None if isinstance(x, float) else x.shape)
    #return val.reshape(-1, 1)
    return val

In [ ]:
def plot_to_fit():
    g = fb.utils.RegularGrid(30, [b[1:] for b in domain_bounds], traverse_order='big')
    X, Y = g.meshgrid(cartesian_index=False)
    Z = g.fun_on_grid(to_fit)
    fp.surface_3D(X, Y, Z, axes_names=('x1', 'x2', 'y'))
plot_to_fit()

In [ ]:
def make_objective(to_fit, sample_num):
    def objective(f):
        bounds = [b[1:] for b in domain_bounds]
        R_g = 0.0
        def g(x):
            return (f(x) - to_fit(x))**2
        # local rewards
        R_ls = []
        print('calculating the local rewards')
        for x in fb.utils.RegularGrid(sample_num, bounds):
            R_l = g(x)
            R_ls.append((x, R_l))
            R_g += R_l
        return R_ls, R_g
    return objective

In [ ]:
class Coordinator(fb.Coordinator):
    def get_pre_phase_config(self, trial_num):
        #c = fb.GPPriorSelectConfig(self.optimiser)
        c = fb.RandomCPSelectConfig(self.optimiser)
        return c

    def get_bayes_config(self, trial_num):
        o = self.optimiser
        c = fb.BayesSelectConfig(o)
        
        n = o.num_available_training_points()
        
        c.surrogate = fb.DistributedGPSurrogate(
            init_params=dict(
                num_experts=10,
                kernel=sk_gp.kernels.WhiteKernel() + 1.0 * sk_gp.kernels.RBF(),
            ),
            optimise_params=dict(
                iterations=3,
                randomize_theta=True,
                extra_starts=2,
                quiet=False
            ),
            predict_params=None,
            parallel=True
        )
        
        """
        c.surrogate = fb.DistributedGPSurrogate(
            init_params=dict(
                num_experts=4,
                kernel=GPy.kern.RBF(o.surrogate_dimensionality()),
                expert_class=dgp.GPyExpert
            ),
            optimise_params=dict(
                iterations=2,
                randomize_theta=True,
                extra_starts=0,
                quiet=False
            ),
            predict_params=None,
            parallel=False
        )
        """
        
        #c.surrogate.optimise_params['num_restarts'] = 1
        #c.surrogate.optimise_params['verbose'] = True
        
        e = fb.IndependentExtractionConfig(o)
        c.extraction_config = e
        e.control_xs = fb.RegularGrid(15, o.domain_bounds)
        e.samples_per_cp = 200
        e.sample_distribution = 'linear'
        
        #e.aux_optimiser_params['num_random'] = 100
        #e.aux_optimiser_params['num_bfgs'] = 0
        e.interpolation = 'cubic'
        
        
        return c

In [ ]:
opt = None
np.random.seed(0)
objective = make_objective(to_fit, sample_num=15)
global opt
opt = fb.Optimiser(objective, domain_bounds, range_bounds, desired_extremum='min')
opt.run(Coordinator(5, 7))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 8))
fp.plot_convergence(opt, optimal_R_g=0, ax=ax1)
#fp.plot_consecutive_distance(opt, ax=ax2)
fp.plot_timings(opt, ax=ax3)
fig.tight_layout()

#fp.plot_trials(opt, opt.trials, to_fit, color_by_reward=True)
#fp.plot_surrogate_with_trials(opt, -1, to_fit)
#
#inc_i, inc = opt.get_incumbent()
#print('incumbent = trial {}'.format(inc_i))
#fp.plot_trial_area(opt, inc, to_fit)

In [ ]:
fp.plot_trial_quantities(opt, {
    'acq_evals': lambda t: t.selection.extraction.acq_evals if t.is_bayes() else 0,
    #'acq_gradient_evals': lambda t: t.selection.extraction.acq_gradient_evals if t.is_bayes() else 0
    #'acq_time': lambda t: t.selection.extraction.acq_total_time/t.selection.extraction.acq_evals if t.is_bayes() else 0,
})
print(opt.trials[-1].selection.extraction.acq_evals)

In [ ]:
g = fb.utils.RegularGrid(100, [(0, 6), (0, 6)])
X, Y = g.meshgrid(cartesian_index=False)
Z = g.fun_on_grid(opt.trials[-1].f)
fp.surface_3D(X, Y, Z, axes_names=('x1', 'x2', 'y'))

In [ ]:
def plot_output_choices(opt, trial_num, x, true_f=None, plot_true_f=False):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    t = opt.trials[trial_num]
    assert type(t.config) == fb.BayesSelectConfig
    
    if x.ndim == 2:
        xmin, xmax = opt.domain_bounds[0]
        ymin, ymax = opt.domain_bounds[1]
        n = 100
        g = fb.utils.RegularGrid(50, [b[1:] for b in domain_bounds], traverse_order='big')
        X, Y = g.meshgrid(cartesian_index=False)
        if plot_true_f:
            assert true_f is not None
            Z = g.fun_on_grid(true_f)
        else:
            Z = g.fun_on_grid(t.f)
        im = ax1.pcolormesh(X, Y, Z, cmap='viridis')
        c = fig.colorbar(im, ax=ax1)
        c.set_label('true_f(x)' if plot_true_f else 'extracted_f(x)')
        ax1.scatter(x[0,0], x[0,1], color='r')
        ax1.set_xlim(xmin, xmax)
        ax1.set_xlabel(opt.domain_names[0])
        ax1.set_ylabel(opt.domain_names[1])
        ax1.set_ylim(ymin, ymax)
    
    ys = np.linspace(*opt.range_bounds, num=100).reshape(-1, 1)
    inputs = opt._points_along_y(x, ys)
    # assuming the extraction method uses an acquisition function
    e = t.config.extraction_config
    acq = e.acquisition(inputs, surrogate=t.surrogate, maximising=opt.is_maximising(), **e.acquisition_params)
    R_mu, R_var = t.surrogate.predict(inputs)
    R_sig = np.sqrt(R_var)
    n_sig = 2
    if true_f is not None:
        ax2.axvline(x=true_f(x), color='k', label='true output')
    ax2.axvline(x=t.f(x), color='C2', linestyle='--', label='extracted function output')
    ax2.plot(ys, R_mu, label='surrogate')
    ax2.plot(ys, acq, label='acquisition')
    ax2.fill_between(ys.flatten(), (R_mu-n_sig*R_sig).flatten(), (R_mu+n_sig*R_sig).flatten(), alpha=0.2)
    ax2.set_xlabel('function output')
    ax2.set_ylabel('predicted local reward')
    ax2.legend()
    
    fig.tight_layout()

def plot_output_choices_interactive(opt, trial_num, true_f=None, plot_true_f=False, **kwargs):
    vals = [kwargs[name] for name in opt.domain_names]
    plot_output_choices(opt, trial_num, np.array([vals]), true_f=true_f, plot_true_f=plot_true_f)

import ipywidgets as widgets

def param_sliders(opt, step=0.25):
    return {name : widgets.FloatSlider(min=xmin, max=xmax, step=step, value=(xmax-xmin)/2)
            for name, (xmin, xmax) in zip(opt.domain_names, opt.domain_bounds)}

widgets.interact(plot_output_choices_interactive, opt=widgets.fixed(opt), trial_num=widgets.fixed(-1), true_f=widgets.fixed(to_fit), plot_true_f=False, **param_sliders(opt));

In [ ]:
print(dir(opt.trials[-1].surrogate.model.experts[0].model))
s = opt.trials[-1].surrogate
#s.model.experts[0].log_marginal_likelihood(s.model.theta)
L = s.model.experts[0].model.L_.copy()
np.fill_diagonal(L, 0)
np.all(np.isclose(L, 0))

In [ ]:
ply.

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
opt.trials[-1].surrogate.model.plot(visible_dims=[0,1], ax=ax)

In [ ]:
opt.get_training_data(fb.SurrogateConfig(opt))[0].shape[0]